## Common formats

### Sequence formats


- `fasta`: plain text records that contain header and sequence (nucleic acid or protein)
- `fastq`: plain text records that contain HTS sequence reads and associated quality scores
- `genebank`: similar to `fasta` but allows annotations (metadata). No standard, very buggy.


### Alignment formats

- `sam` plain text records that contain read and alignment info
- `bam` binary (compressed) form of `sam`
- `pileup` tab delimited; records contain aggregate alignment data per reference position

### Position (genomic interval) formats

- `gff` (former `gtf`) genomic feature format; tab-delimited plain text
- `bed` generic position format